In [1]:
using VerilogWriter

## Examples in Documents

In [2]:
let
    prs = @parameters (dummy = 10)

    ps = @ports (
        @in CLK, RST, sig1, sig2;
        @in 8 din;
        @out @reg 8 dout
    )

    ds = @decls (
        @reg 8 dbuf
    )

    proc = @always (
        if sig1
            dbuf <= din 
        elseif sig2 
            dout <= dbuf
        else
            dout <= ~din 
        end
    )

    mymod = Vmodule(
        "mymodule",
        prs,
        ps,
        ds,
        Assign[],
        [autoreset(proc)]
    )

    vshow(mymod)
end

module mymodule #(
    parameter dummy = 10
)(
    input CLK,
    input RST,
    input sig1,
    input sig2,
    input [7:0] din,
    output reg [7:0] dout
);
    reg [7:0] dbuf;

    always_ff @( posedge CLK ) begin
        if (RST) begin
            dbuf <= 0;
            dout <= 0;
        end else begin
            if (sig1) begin
                dbuf <= din;
            end else if (sig2) begin
                dout <= dbuf;
            end else begin
                dout <= ~din;
            end
        end
    end
endmodule
type: Vmodule


### Design Full Module on VerilogWriter.jl

In [3]:
function main()
    # AXI master, no `multiple outstanding addresses`
    prms = @parameters (
        DWID = 32
    )

    mawport = @ports (
        @out @reg awvalid;
        @in awready;
        @out @reg 32 awaddr;
        @out @reg 3 awprot
    )
    mwport = @ports (
        @out @reg wvalid;
        @in wready;
        @out @reg DWID wdata;
        @out @reg DWID >> 3 wstrb
    )
    mbport = @ports (
        @in bvalid;
        @out @reg bready;
        @in 2 bresp
    )

    mwportall = declmerge(mawport, mwport, mbport)
    
    dport = @ports (
        @in CLK, RST;
        @in dvalid;
        @in DWID din;
        @in 32 daddr;
        @out @reg ddone
    )

    wfsm = @FSM wstate widle, waready, wdready, wfready
    
    bfsmcond = @always (
        i2a0 = awvalid & awready;
        i2d0 = wvalid & wready;
        i2a = i2a0 & ~i2d0;
        i2d = ~i2a0 & i2d0;
        i2r = i2a0 & i2d0;
        a2r = i2d0;
        d2r = i2a0;
        r2i = bvalid & bready
    )

    @sym2wire i2a, i2d, i2r, a2r, d2r, r2i

    transadd!(
        wfsm, 
        [
            (i2a, @tstate widle => waready),
            (i2d, @tstate widle => wdready),
            (i2r, @tstate widle => wfready),
            (a2r, @tstate waready => wfready),
            (d2r, @tstate wdready => wfready),
            (r2i, @tstate wfready => widle)
        ]
    )

    bports = @always (
        ddone = wstate == wfready;

        awvalid = dvalid;
        awaddr = daddr;
        awprot = 0;

        wvalid = dvalid;
        wdata = din;
        wstrb = ~0;

        bready = 1
    )

    env = Vmodenv(
        prms, 
        declmerge(dport, mwportall),
        fsmconv(Localparams, wfsm),
        Decls(fsmconv(Onedecl, wfsm))
    )

    avec = [
        autoreset(fsmconv(Alwayscontent, wfsm)), 
        bfsmcond, 
        bports
    ]

    d, nenv = autodecl(avec, env)

    vmod = Vmodule(
        "myaxim",
        nenv.prms,
        nenv.prts,
        nenv.lprms, 
        declmerge(d, nenv.dcls),
        Assign[],
        avec
    )

    open("myaxim.v", "w") do io 
        write(io, string(vmod, false))
    end
end

@time main()

  0.373056 seconds (777.49 k allocations: 42.220 MiB, 15.76% gc time, 99.51% compilation time)


2129

Code above exports the following file.

```verilog
module myaxim #(
    parameter DWID = 32
)(
    input CLK,
    input RST,
    input dvalid,
    input [DWID-1:0] din,
    input [31:0] daddr,
    output reg ddone,
    output reg awvalid,
    input awready,
    output reg [31:0] awaddr,
    output reg [2:0] awprot,
    output reg wvalid,
    input wready,
    output reg [DWID-1:0] wdata,
    output reg [(DWID >> 3)-1:0] wstrb,
    input bvalid,
    output reg bready,
    input [1:0] bresp
);
    localparam widle = 0;
    localparam waready = 1;
    localparam wdready = 2;
    localparam wfready = 3;

    reg a2r;
    reg d2r;
    reg i2a;
    reg i2a0;
    reg i2d;
    reg i2d0;
    reg i2r;
    reg r2i;
    reg [1:0] wstate;

    always @( posedge CLK ) begin
        if (RST) begin
            wstate <= 0;
        end else begin
            case (wstate)
                widle: begin
                    if (i2a) begin
                        wstate <= waready;
                    end else if (i2d) begin
                        wstate <= wdready;
                    end else if (i2r) begin
                        wstate <= wfready;
                    end
                end
                waready: begin
                    if (a2r) begin
                        wstate <= wfready;
                    end
                end
                wdready: begin
                    if (d2r) begin
                        wstate <= wfready;
                    end
                end
                wfready: begin
                    if (r2i) begin
                        wstate <= widle;
                    end
                end
            endcase
        end
    end
    always @* begin
        i2a0 = (awvalid & awready);
        i2d0 = (wvalid & wready);
        i2a = (i2a0 & ~i2d0);
        i2d = (~i2a0 & i2d0);
        i2r = (i2a0 & i2d0);
        a2r = i2d0;
        d2r = i2a0;
        r2i = (bvalid & bready);
    end
    always @* begin
        ddone = (wstate == wfready);
        awvalid = dvalid;
        awaddr = daddr;
        awprot = 0;
        wvalid = dvalid;
        wdata = din;
        wstrb = ~0;
        bready = 1;
    end
endmodule
```